In [1]:
import numpy as np
import stochpy
import matplotlib.pyplot as plt
import pickle
from scipy.interpolate import interp1d


#######################################################################
#                                                                     #
#            Welcome to the interactive StochPy environment           #
#                                                                     #
#######################################################################
#  StochPy: Stochastic modeling in Python                             #
#  http://stochpy.sourceforge.net                                     #
#  Copyright(C) T.R Maarleveld, B.G. Olivier, F.J Bruggeman 2010-2025 #
#  DOI: 10.1371/journal.pone.0079345                                  #
#  Email: tmd200@users.sourceforge.net                                #
#  VU University, Amsterdam, Netherlands                              #
#  Centrum Wiskunde Informatica, Amsterdam, Netherlands               #
#  StochPy is distributed under the BSD licence.                      #
###############################################################

In [ ]:
mu = 100
ref_price_a = ref_price_b = 100
std_price_a = std_price_b = 1

price_bins = np.linspace(
    min(ref_price_a, ref_price_b) - 5 * std_price_b,
    max(ref_price_a, ref_price_b) + 5 * std_price_a,
    100
)

# Midpoints and bin width
midpoints = (price_bins[:-1] + price_bins[1:]) / 2
dp = midpoints[1] - midpoints[0]
num_bins = len(midpoints)  

prices_A = midpoints.copy()
prices_B = midpoints.copy() 

# Parámetros de simulación
phi = 20 * dp
delta = 0.01
kappa = 100000
beta = 70

species_A = [f"A{i+1}" for i in range(num_bins)]
species_B = [f"B{i+1}" for i in range(num_bins)]

all_species = species_A + species_B

def gaussian(x):
    return (1/(std_price_a * np.sqrt(2 * np.pi))) * np.exp(- (x - mu)**2 / (2 * std_price_a**2))

gaussian_values = [gaussian(p) for p in prices_A]

In [3]:
with open("C:\\Users\\A\\Documents\\Investigacion\\Tesis\\ipython_notebooks\\Codigos finales para la tesis\\resources\\Gillespie with beta\\initial_n_A.pkl", 'rb') as f:
    initial_A = pickle.load(f)
with open("C:\\Users\\A\\Documents\\Investigacion\\Tesis\\ipython_notebooks\\Codigos finales para la tesis\\resources\\Gillespie with beta\\initial_n_B.pkl", 'rb') as f:
    initial_B = pickle.load(f)

In [4]:
creation_reactions = []
for i in range(num_bins):
    # Reacciones de creación para A
    rate_A = phi * gaussian_values[i]
    rate_A_str = "{:.10f}".format(rate_A)
    creation_reactions.append(
        f"creation_{species_A[i]}:\n"
        f"    $pool > {species_A[i]}\n"
        f"    {rate_A_str}"
    )
    # Reacciones de creación para B
    rate_B = phi * gaussian_values[i]
    rate_B_str = "{:.10f}".format(rate_B)
    creation_reactions.append(
        f"creation_{species_B[i]}:\n"
        f"    $pool > {species_B[i]}\n"
        f"    {rate_B_str}"
    )

death_reactions = []
for i in range(num_bins):
    death_reactions.append(
        f"death_{species_A[i]}:\n"
        f"    {species_A[i]} > $pool\n"
        f"    delta * {species_A[i]}"
    )
    death_reactions.append(
        f"death_{species_B[i]}:\n"
        f"    {species_B[i]} > $pool\n"
        f"    delta * {species_B[i]}"
    )

interaction_reactions = []
for A_idx in range(num_bins):
    for B_idx in range(A_idx + 1, num_bins):
        A_species = species_A[A_idx]
        B_species = species_B[B_idx]
        if prices_B[B_idx] >= prices_A[A_idx]:
            price_diff = prices_B[B_idx] - prices_A[A_idx]
            rate = np.exp(beta * price_diff) * kappa
            rate_str = f"{rate:.10f}"
            reaction = (
                f"interaction_{A_species}_{B_species}:\n"
                f"    {A_species} + {B_species} > $pool\n"
                f"    {rate_str} * {A_species} * {B_species}"
            )
            interaction_reactions.append(reaction)

all_reactions = creation_reactions + death_reactions + interaction_reactions

# Escribir modelo
model_filename = "Gillespie_gas_con_beta_final.psc"
full_model_path = "C:\\Stochpy\\pscmodels\\Gillespie_gas_con_beta_final.psc"

with open(full_model_path, 'w') as f:
    # Condiciones iniciales
    f.write("#InitVar:\n")
    for i in range(num_bins):
        f.write(f"{species_A[i]} = {initial_A[i]}\n")
    for i in range(num_bins):
        f.write(f"{species_B[i]} = {initial_B[i]}\n")
    f.write("\n")
    
    # Parámetros
    f.write("#InitPar:\n")
    f.write(f"delta = {delta}\n")
    f.write("\n")
    
    # Reacciones
    f.write("#Reactions:\n")
    for reaction in all_reactions:
        f.write(reaction + "\n\n")

model = stochpy.SSA()
model.Model(full_model_path)

In [ ]:
desired_order =  species_A + species_B
num_sims = 5
model.Endtime(1500)

model.DoStochSim(
    trajectories = num_sims,
    species_selection=desired_order 
)

In [ ]:
total_a_sim_array = []
total_b_sim_array = []
stationary_a_sim_array = []
stationary_b_sim_array = []

for i in range(num_sims):

    model.GetTrajectoryData(i+1)

    total_timepoints = len(model.data_stochsim.time)

    percent = 0.5
    last_percent = int(percent * total_timepoints)
    start_idx = max(0, total_timepoints - last_percent)

    counts_A = model.data_stochsim.species[:,:num_bins]
    counts_B = model.data_stochsim.species[:,num_bins:]

    final_A_counts = np.mean(counts_A[start_idx:, :], axis=0) / dp
    final_B_counts = np.mean(counts_B[start_idx:, :], axis=0) / dp

    total_particles_A = []
    total_particles_B = [] 

    times = model.data_stochsim.time

    for t in range(total_timepoints):

        counts_Ai = counts_A[t]
        counts_Bi = counts_B[t]

        total_particles_A.append(np.sum(counts_Ai)) 
        total_particles_B.append(np.sum(counts_Bi))

    total_particles_A = np.column_stack((total_particles_A,times))
    total_particles_B = np.column_stack((total_particles_B,times))
        
    total_a_sim_array.append(total_particles_A)
    total_b_sim_array.append(total_particles_B)
    stationary_a_sim_array.append(final_A_counts)
    stationary_b_sim_array.append(final_B_counts)

dt = 1  
common_time = np.arange(0, 1500 + dt, dt)

def interpolate_series(times, values, common_time):
    
    times, idx = np.unique(times, return_index=True)
    values = values[idx]

    interp_func = interp1d(times, values, kind='linear', fill_value='extrapolate')
  
    return interp_func(common_time)

all_interpolated_a = [] 

for sim_data in total_a_sim_array:
    times = sim_data[:,1]
    values = sim_data[:,0]
    interpolated = interpolate_series(times, values, common_time)
    all_interpolated_a.append(interpolated)

all_interpolated_a = np.array(all_interpolated_a)
total_a_sim_array = np.mean(all_interpolated_a, axis=0)

all_interpolated_b = []

for sim_data in total_b_sim_array:
    times = sim_data[:,1]
    values = sim_data[:,0]
    interpolated = interpolate_series(times, values, common_time)
    all_interpolated_b.append(interpolated)

all_interpolated_b = np.array(all_interpolated_b)
total_b_sim_array = np.mean(all_interpolated_b, axis=0)

mean_stationary_a_sim_array = np.mean(np.array(stationary_a_sim_array), axis = 0)
mean_stationary_b_sim_array = np.mean(np.array(stationary_b_sim_array), axis = 0)

with open("resources\\Gillespie with beta\\n_A_totales_gillespie.pkl", 'wb') as f:
    pickle.dump(total_a_sim_array,f)
with open("resources\\Gillespie with beta\\n_B_totales_gillespie.pkl", 'wb') as f:
    pickle.dump(total_b_sim_array,f)
with open("resources\\Gillespie with beta\\times_gillespie.pkl", 'wb') as f:
    pickle.dump(common_time,f)
with open("resources\\Gillespie with beta\\n_A_estacionario_gillespie.pkl", 'wb') as f:
    pickle.dump(mean_stationary_a_sim_array,f)
with open("resources\\Gillespie with beta\\n_B_estacionario_gillespie.pkl", 'wb') as f:
    pickle.dump(mean_stationary_b_sim_array,f)

In [ ]:
with open("resources\\Gillespie mercado informal\\n_A.pkl", 'wb') as f:
    pickle.dump(total_particles_A,f)
with open("resources\\Gillespie mercado informal\\n_B.pkl", 'wb') as f:
    pickle.dump(total_particles_B,f)
with open("resources\\Gillespie mercado informal\\times.pkl", 'wb') as f:
    pickle.dump(times,f)
with open("resources\\Gillespie mercado informal\\n_A_estacionario.pkl", 'wb') as f:
    pickle.dump(final_A_counts,f)
with open("resources\\Gillespie mercado informal\\n_B_estacionario.pkl", 'wb') as f:
    pickle.dump(final_B_counts,f)
with open("resources\\Gillespie mercado informal\\initial_A.pkl", 'wb') as f:
    pickle.dump(initial_A,f)
with open("resources\\Gillespie mercado informal\\initial_B.pkl", 'wb') as f:
    pickle.dump(initial_B,f)